In [ ]:
import pandas as pd
import seaborn as sns

path_main_dir = ".."

df = pd.read_csv(path_main_dir + "/output/results.csv")
df.head()

In [ ]:
# ax = sns.boxplot(x="cleora_n_iter", y="log_loss", hue="algo",
#                  data=df, palette="Set3")

In [ ]:
default_params = {
    #embedding computation
    'cleora_n_iter': 5,
    'cleora_dim': 1024,

    #dataset preparation
    'train_test_split': 0.2,
    'dataset_name': 'fb-pages',
    #options: fb-pages, walmart-trips, trivago-clicks, trivago-clicks-continents
    #modifiers: -resampled-node/-resampled-hyperedge for all datasets

    'batch_size': 256,
    'test_batch_size': 1000,
    'epochs': 20,
}

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib

os.makedirs(f"{path_main_dir}/output/plots/experiments", exist_ok=True)
matplotlib.rcParams.update({'font.size': 12})

for dataset_name in ['fb-pages', 'walmart-trips', 'trivago-clicks-continents']:
    default_params['dataset_name'] = dataset_name
    for param in ['cleora_n_iter', 'cleora_dim']:
        data = df
        params = default_params.copy()
        params.pop(param)

        for k, v in params.items():
            data = data.loc[data[k] == v]

        data.sort_values(by=[param], inplace=True)

        fig, ((ax11, ax21), (ax12, ax22)) = plt.subplots(2, 2, sharex=True)

        fig.set_size_inches((12, 6))
        fig.suptitle(params["dataset_name"])
        fig.supxlabel(param)
        fig.supylabel('value')

        dct = {'clique': (ax11, ax12), 'star': (ax21, ax22)}

        for algo, (ax1, ax2) in dct.items():
            ax1.set_title(algo)
            # ax1.set_xticks(list(data.loc[data['algo'] == algo][param]))

            data_final = data.loc[data['algo'] == algo]

            for measure in ['f1_micro', 'f1_macro']:
                ax1.errorbar(param, measure, yerr=f"std_{measure}", label=measure, data=data_final)
            ax2.errorbar(param, "log_loss", yerr=f"std_log_loss", label="log_loss", color='r', data=data_final)

            ax1.legend()
            ax2.legend()

        plt.tight_layout()
        plt.savefig(f"{path_main_dir}/output/plots/experiments/{dataset_name}_{param}")
        plt.show()

In [ ]:
# experiments = df.to_dict()
# [experiments.pop(k) for k in ["epochs", "train_test_split", "batch_size", "test_batch_size"]]
#
# # gmean_scores = []
# # for key in logs.keys():
# #     gmean_scores.append(logs[key]['gmean'])
#
# plot_colors = ['blue', 'green', 'orange', 'violet', 'red', 'brown', 'pink', 'orange', 'violet', 'red', 'brown', 'pink']
# fig, axs = plt.subplots(len(experiments.keys()) + 1, **{"figsize": (6.5, 6.5)})
# fig.suptitle("a")
# axs[-1].set(xlabel="Experiment Number")
#
# axs[0].plot(range(len(experiments["algo"].keys())), gmean_scores, plot_colors[0], markersize=5, marker='o')
# axs[0].set(ylabel="Geometric Mean Score")
# axs[0].grid(True)
#
# for key in experiments.keys():
#     param_values = []
#     for exp in parameters_perm:
#         param_values.append(exp[key])
#
#     plot_index = list(parameters_dist.keys()).index(key) + 1
#     ax = axs[plot_index]
#     ax.plot(range(1, len(param_values) + 1), param_values, plot_colors[plot_index], markersize=5, marker='o')
#     ax.set(ylabel=key)
#     ax.grid(True)
#
# plt.savefig(filepath_for_logs_and_plots + filename + ".png")
# plt.show()